In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

import os
import requests
import json
from dotenv import load_dotenv


import geopandas as gpd

First of all 

In [2]:
client = MongoClient("localhost:27017")

In [3]:
db = client["ironhack"]

### Function

In [4]:
def get_dataframe(collection, query): #Collection admits string, query admits dictionray with filter,projection.
    c = db.get_collection(collection)
    filter_ = query[0]
    projection = query[1]
    list_ = list(c.find(filter_, projection))
    df = pd.DataFrame(list_).explode("offices").reset_index(drop=True)
    df = pd.concat([df, df["offices"].apply(pd.Series)], axis=1).reset_index(drop=True)
    df = df.drop(["offices", 0], axis = 1)
    return df

### Video-Games

In [5]:
query_games = [{"category_code":"games_video"},{"_id": 0, "name":1, "category_code":1, "total_money_raised":1, "offices.city":1, "offices.state_code": 1, "offices.country_code":1, "offices.latitude": 1, "offices.longitude": 1}]

In [6]:
games_df = get_dataframe("companies", query_games)

In [7]:
games_df.head()

,name,category_code,total_money_raised,city,state_code,country_code,latitude,longitude
0,Flektor,games_video,$0,Culver City,CA,USA,34.025958,-118.379768
1,Lala,games_video,$44.2M,Palo Alto,CA,USA,37.451151,-122.154369
2,Joost,games_video,$45M,New York,NY,USA,40.746497,-74.009447
3,Babelgum,games_video,$13.2M,London,None,GBR,53.344104,-6.267494
4,Sparter,games_video,$0,None,None,USA,37.090240,-95.712891


### More than 1M founding

In [8]:
query_funded = [{"total_money_raised" : {"$regex" : "[$€].*[MB]"}},{"_id": 0, "name":1, "category_code":1, "total_money_raised":1, "offices.city":1, "offices.state_code": 1, "offices.country_code":1, "offices.latitude": 1, "offices.longitude": 1}]

In [9]:
funded_df = get_dataframe("companies", query_funded )

In [10]:
funded_df

,name,category_code,total_money_raised,city,state_code,country_code,latitude,longitude
0,Wetpaint,web,$39.8M,Seattle,WA,USA,47.603122,-122.333253
1,Wetpaint,web,$39.8M,New York,NY,USA,40.723731,-73.996431
2,Digg,news,$45M,San Francisco,CA,USA,37.764726,-122.394523
3,Facebook,social,$2.43B,Menlo Park,CA,USA,37.416050,-122.151801
4,Facebook,social,$2.43B,Dublin,None,IRL,53.344104,-6.267494
...,...,...,...,...,...,...,...,...
5289,Celestial Semiconductor,semiconductor,$2M,San Jose,CA,USA,37.386417,-121.929170
5290,Purfresh,cleantech,$62.7M,Menlo Park,CA,USA,37.477008,-122.151599
5291,InSound Medical,biotech,$11M,Newark,CA,USA,37.512038,-121.998513
5292,Getyoo,mobile,€1.13M,Brussels,None,BEL,50.839970,4.346472


### Design

In [11]:
query_design = [{"tag_list": {"$regex" : "design"}}, {"_id": 0, "name":1, "category_code":1, "total_money_raised":1, "offices.city":1, "offices.state_code": 1, "offices.country_code":1, "offices.latitude": 1, "offices.longitude": 1}]

In [12]:
design_df = get_dataframe("companies", query_design )

In [13]:
design_df.head()

,name,category_code,total_money_raised,city,state_code,country_code,latitude,longitude
0,Digg,news,$45M,San Francisco,CA,USA,37.764726,-122.394523
1,Adobe Systems,software,$0,San Jose,CA,USA,37.320052,-121.877636
2,Negonation,web,$0,,None,ESP,40.463667,-3.749220
3,GENWI,mobile,$7.1M,Los Altos,CA,USA,33.817100,-111.903500
4,Aviary,mobile,$19M,NY,NY,USA,NaN,NaN


## Find countries that have all 3 types of companies

### Countries with Gaming Companies

In [14]:
games_df_countries = games_df.groupby(['country_code']).count().sort_values("name",ascending = False)

In [15]:
games_df_countries.head(10)

,name,category_code,total_money_raised,city,state_code,latitude,longitude
country_code,,,,,,,
USA,587,587,587,580,570,464,464
GBR,74,74,74,74,1,36,36
CAN,35,35,35,34,0,24,24
DEU,30,30,30,30,0,21,21
FRA,26,26,26,26,0,20,20
ISR,20,20,20,19,0,9,9
CHN,17,17,17,17,0,1,1
IND,15,15,15,15,0,8,8
JPN,14,14,14,13,0,1,1


**Countries with Companies with over 1M in funding**

In [16]:
funded_df_countries = funded_df.groupby(['country_code']).count().sort_values("name",ascending=False)

In [17]:
funded_df_countries.head(10)

,name,category_code,total_money_raised,city,state_code,latitude,longitude
country_code,,,,,,,
USA,3548,3503,3548,3522,3492,2674,2674
GBR,291,290,291,291,7,113,113
CAN,137,136,137,137,0,92,92
ISR,132,131,132,129,3,25,25
FRA,127,126,127,127,0,76,76
DEU,107,107,107,107,1,63,63
IND,96,95,96,96,3,36,36
CHN,96,95,96,95,0,10,10
AUS,46,46,46,46,0,21,21


**Countries with Design Companies**

In [18]:
design_df_countries = design_df.groupby(['country_code']).count().sort_values("name",ascending=False)

In [19]:
design_df_countries.head(10)

,name,category_code,total_money_raised,city,state_code,latitude,longitude
country_code,,,,,,,
USA,350,347,350,350,346,285,285
GBR,69,68,69,69,0,33,33
IND,57,57,57,57,1,20,20
CAN,22,22,22,22,0,17,17
AUS,16,16,16,16,0,13,13
DEU,12,12,12,12,0,12,12
NLD,11,11,11,11,0,9,9
ARG,11,11,11,11,0,6,6
FRA,10,10,10,10,0,7,7


In all 3 categories USA is clear ahead of any other country, so to select the cities we will analyze USA separatedly

The countries that I will select are USA, Canada, United Kingdom and Spain. 

In order to select the cities that we will analyze, we will take 2 cities from USA, and one for each of the other selected countries.

### City Selection USA

In [20]:
def find_cities (df,countries):
    selected_df = df.loc[df['country_code'].isin(countries)]
    selected_df = selected_df.groupby(['city']).count().sort_values("name",ascending=False).head(10)
    return selected_df

In [21]:
cities_games_us = find_cities (games_df, ["USA"])

In [22]:
cities_games_us.head()

,name,category_code,total_money_raised,state_code,country_code,latitude,longitude
city,,,,,,,
New York,75,75,75,73,75,51,51
San Francisco,68,68,68,68,68,49,49
Los Angeles,31,31,31,31,31,25,25
,24,24,24,17,24,16,16
Palo Alto,16,16,16,16,16,16,16


In [23]:
cities_funded_us = find_cities (funded_df, ["USA"])

In [24]:
cities_funded_us.head()

,name,category_code,total_money_raised,state_code,country_code,latitude,longitude
city,,,,,,,
San Francisco,436,432,436,436,436,318,318
New York,333,328,333,333,333,221,221
Seattle,110,108,110,110,110,88,88
Mountain View,106,106,106,106,106,90,90
Palo Alto,104,102,104,104,104,84,84


In [25]:
cities_design_us = find_cities(design_df, ["USA"])

In [26]:
cities_design_us.head()

,name,category_code,total_money_raised,state_code,country_code,latitude,longitude
city,,,,,,,
New York,23,23,23,23,23,18,18
San Francisco,20,20,20,20,20,12,12
San Diego,11,8,11,11,11,8,8
Los Angeles,10,10,10,10,10,9,9
,9,9,9,7,9,3,3


It is crystal clear that the 2 cities with best results in both aspects are New York And San Francisco, so these will be the cities that we will consider in the USA.

### City Selection Spain, UK and Canada

**Gaming companies**

In [27]:
cities_games_rest = find_cities(games_df,["GBR", "CAN", "ESP"])

In [28]:
cities_games_rest.head(10)

,name,category_code,total_money_raised,state_code,country_code,latitude,longitude
city,,,,,,,
London,35,35,35,0,35,19,19
,7,7,7,0,7,2,2
Vancouver,7,7,7,0,7,7,7
Toronto,6,6,6,0,6,5,5
Madrid,6,6,6,0,6,5,5
Oxford,4,4,4,0,4,2,2
Montreal,3,3,3,0,3,2,2
Corby,3,3,3,0,3,3,3
Dundee,3,3,3,1,3,2,2


**Companies with funding**

In [29]:
cities_funded_rest = find_cities(funded_df,["GBR", "CAN", "ESP"])

In [30]:
cities_funded_rest.head(10)

,name,category_code,total_money_raised,state_code,country_code,latitude,longitude
city,,,,,,,
London,167,166,167,2,167,79,79
Toronto,23,23,23,0,23,17,17
Madrid,16,16,16,0,16,12,12
Barcelona,16,16,16,1,16,8,8
Vancouver,16,16,16,0,16,14,14
,13,13,13,0,13,4,4
Montreal,13,13,13,0,13,11,11
Cambridge,10,10,10,0,10,1,1
"Toronto, Ontario",8,8,8,0,8,5,5


**Design Companies**

In [31]:
cities_design_rest = find_cities(design_df,["GBR", "CAN", "ESP"])

In [32]:
cities_design_rest.head(30)

,name,category_code,total_money_raised,state_code,country_code,latitude,longitude
city,,,,,,,
London,24,24,24,0,24,14,14
,5,5,5,0,5,3,3
Toronto,4,4,4,0,4,2,2
Montreal,3,3,3,0,3,3,3
Birmingham,3,3,3,0,3,3,3
Vancouver,3,3,3,0,3,1,1
Lancaster,2,2,2,0,2,1,1
Leicester,2,2,2,0,2,0,0
Glasgow,2,2,2,0,2,0,0


In [34]:
games_df["type"] = "videogame"

In [35]:
funded_df["type"] = "funded"

In [36]:
design_df["type"] = "design"

In [37]:
def concat_tables_city(city):
    merged = pd.concat([games_df,funded_df,design_df])
    merged_city = merged[merged["city"] == city]
    merged_city = merged_city[merged_city['latitude'].notna()]
    merged_city = merged_city[merged_city['longitude'].notna()]
    return merged_city

In [152]:
def map_city(df, city_lat, city_long):
    city_map = folium.Map(location = [city_lat, city_long], zoom_start = 12)
    for index, row in df.iterrows():
    
        #1. MARKER without icon
        geo_dict = {"location": [row["latitude"], row["longitude"]], "tooltip": row["type"]}

        #2. Icon
        if row["type"] == "videogame":        
            icon = Icon (
                color="blue",
                opacity = 0.6,
                prefix = "fa",
                icon="gamepad",
                icon_color = "black"
            )
        elif row["type"] == "funded":
            icon = Icon (
                color="red",
                opacity = 0.6,
                prefix = "fa",
                icon="money",
                icon_color = "white"
            )
        elif row["type"] == "design":
            icon = Icon (
                color="darkpurple",
                opacity = 0.6,
                prefix = "fa",
                icon="pencil",
                icon_color = "white"
            )
        else:
            icon = Icon (
                color="blue",
                opacity = 0.6,
                prefix = "fa",
                icon="question",
                icon_color = "brown",
                icon_size=(14, 14)
            )
        #3. Marker
        new_marker = Marker(**geo_dict, icon = icon, radius = 2)

        #4. Add the Marker
        new_marker.add_to(city_map)
    return city_map

### New York

In [153]:
new_york_df = concat_tables_city("New York")

In [154]:
ny_map = map_city(new_york_df, 40.7477748,-73.9916559)

In [155]:
ny_map

### Toronto

In [156]:
toronto_df = concat_tables_city("Toronto")

In [157]:
toronto_map = map_city(toronto_df, 43.6724094, -79.4677423)

In [158]:
toronto_map

### London

In [159]:
london_df = concat_tables_city("London")

In [160]:
london_map = map_city(london_df, 51.5285582,-0.2416799 )

In [161]:
london_map

### Barcelona

In [162]:
barcelona_df = concat_tables_city("Barcelona")

In [163]:
barcelona_map = map_city(barcelona_df, 41.3925788, 1.9966654)

In [164]:
barcelona_map

In [51]:
load_dotenv()

True

In [52]:
token_fsq = os.getenv("token_foursquare")

In [53]:
def get_results_from_foursquare (query, location, limit=1):
    
    # Doing the call for foursquare     
    ll = f"{location[0]}%2C{location[1]}"
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&limit={str(limit)}"

    headers = {
        "accept": "application/json",
        "Authorization": token_fsq,
    }
    
    response = requests.get(url, headers=headers).json()
    
    return response["results"][0]["distance"]

In [54]:
def get_distances(location):
    places = ["starbucks", "school", "nightclub", "basketball", "vegan", "airport"]
    distances = []
    for i in places:
        distances.append(get_results_from_foursquare(i,location))
    return distances

In [171]:
def add_icon(map_,location):
    geo_dict = {"location": [float(location[0]), float(location[1])], "tooltip": "Selected Location"}

    icon = Icon (
    color="green",
    opacity = 0.6,
    prefix = "fa",
    icon="exclamation",
    icon_color = "black")
        
    new_marker = Marker(**geo_dict, icon = icon, radius = 2)

    new_marker.add_to(map_)
        
    return map_

### Barcelona

In [172]:
location_bcn = ["41.4002552", "2.1773209"]

In [173]:
barcelona_upd_map = add_icon(barcelona_map, location_bcn)

In [174]:
barcelona_upd_map

In [96]:
Barcelona = get_distances(location_bcn)

In [97]:
Barcelona

[1280, 3154, 1198, 6541, 2158, 14365]

In [98]:
get_results_from_foursquare ("escuela", location_bcn)

659

In [99]:
Barcelona[1] = 659

In [100]:
Barcelona

[1280, 659, 1198, 6541, 2158, 14365]

In [122]:
Barcelona

['Barcelona', 1280, 659, 1198, 6541, 2158, 14365]

In [123]:
Barcelona[1]=358

### Toronto

In [101]:
location_toronto = ["43.6680092","-79.3929092"]

In [175]:
toronto_upd_map = add_icon(toronto_map, location_toronto)

In [176]:
toronto_upd_map

In [102]:
Toronto = get_distances(location_toronto)

In [103]:
Toronto

[366, 4202, 4305, 2948, 2232, 17730]

### London

In [104]:
location_london = ["51.5217003","-0.1122619"]

In [177]:
london_upd_map = add_icon(london_map, location_london)

In [178]:
london_upd_map

In [105]:
London = get_distances(location_london)

In [106]:
London

[947, 881, 2309, 2860, 1926, 24665]

### New York

In [107]:
location_newyork = ["40.7458636","-73.9919514"]

In [180]:
newyork_upd_map = add_icon(ny_map, location_newyork)

In [181]:
newyork_upd_map

In [108]:
NewYork = get_distances(location_newyork)

In [109]:
NewYork

[373, 2824, 1504, 1824, 5994, 891]

In [110]:
NewYork[5] = 21000

In [111]:
NewYork

[373, 2824, 1504, 1824, 5994, 21000]

In [112]:
Barcelona.insert(0, "Barcelona")

In [113]:
Toronto.insert(0, "Toronto")

In [114]:
London.insert(0, "London")

In [115]:
NewYork.insert(0, "New York")

In [116]:
data = [Barcelona, Toronto, London, NewYork]

In [117]:
data

[['Barcelona', 1280, 659, 1198, 6541, 2158, 14365],
 ['Toronto', 366, 4202, 4305, 2948, 2232, 17730],
 ['London', 947, 881, 2309, 2860, 1926, 24665],
 ['New York', 373, 2824, 1504, 1824, 5994, 21000]]

In [124]:
city_distances = pd.DataFrame(data, columns = ["City", "Starbucks", "School", "Nightclub", "Basketball", "Vegan", "Airport"])

In [125]:
city_distances

,City,Starbucks,School,Nightclub,Basketball,Vegan,Airport
0,Barcelona,358,659,1198,6541,2158,14365
1,Toronto,366,4202,4305,2948,2232,17730
2,London,947,881,2309,2860,1926,24665
3,New York,373,2824,1504,1824,5994,21000


In [126]:
city_distances["Score"] = (100/city_distances["Starbucks"]*0.2)+(100/city_distances["School"]*0.2)+(100/city_distances["Nightclub"]*0.15)+(100/city_distances["Basketball"]*0.05)+(100/city_distances["Vegan"]*0.15)+(100/city_distances["Airport"]*0.25)

In [127]:
city_distances

,City,Starbucks,School,Nightclub,Basketball,Vegan,Airport,Score
0,Barcelona,358,659,1198,6541,2158,14365,0.108191
1,Toronto,366,4202,4305,2948,2232,17730,0.072715
2,London,947,881,2309,2860,1926,24665,0.060867
3,New York,373,2824,1504,1824,5994,21000,0.077109
